In [9]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data, datasets # 자연어셋을 다루기 때문에 토치비전이 아닌 토치텍스트 사용 

In [10]:
BATCH_SIZE = 64
lr = 0.001
EPOCHS = 40
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

신경망 학습에 사용될 IMDB 데이터셋을 로딩하고 텐서로 변환한다.
먼저 텍스트 형태의 영화 리뷰들과 그에 해당하는 레이블을 텐서로 바꿀 때 필요한 설정을 정한다.
토치 텍스트를 이용하면 이러한 설정 정보를 담은 TEXT와 LABEL  이라는 객체를 쉽게 생성할 수 있다. 

sequentia 파라미터 사용 -> 데이터 셋이 순차적인 데이터셋인지 명시 
레이블 값은 단순히 클래스를 나타내는 숫자로 순차적인  데이터가 아님 
batch_first 파라미터로 신경망에 입력되는 텐서의 첫 번째 차원값이 batch_size 가 되도록 정한다.
lower 변수를 이용해 텍스트의 데이터를 모두 소문자화 

split() 함수를 이용해 모델에 입력되는 데이터 셋 생성 
학습셋인 trainset과 테스트셋인 testset 생성 

In [ ]:
TEXT = data.Field(sequential = True, batch_first = True, lower = True)
LABEL = data.Field(sequential = False, batch_first = True)

trainset, testset = datasets.IMDB.splits(TEXT, LABEL)

데이터셋을 이용해 워드 임베딩에 필요한 단어 사전을 만든다. 
min_freq 는 학습 데이터에서 최소 5번이상 등장한 단어만을 사전에 담음 
학습 데이터에서 5번 미만으로 출현하는 단어는 'unk' 토큰으로 대체 

In [12]:
TEXT.build_vocab(trainset, min_freq = 5)
LABEL.build_vocab(trainset)

검증셋을 만든다. 
IMDB 데이터셋에선 따로 검증셋이 없기 때문에 학습셋의 일부를 사용 
trainset.split( split_ratio = 0.8) -> trainset 80 % valset 20%


In [13]:
trainset, valset = trainset.split ( split_ratio = 0.8 )
# 배치 설정 
train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (trainset, valset, testset),
    batch_size = BATCH_SIZE,
    shuffle = True, repeat = False
)

In [14]:
vocab_size = len(TEXT.vocab)
n_classes = 2  

print (" [학습셋] : %d [검증셋] : %d [테스트셋] : %d [단어수] : %d [클래스] : %d "%(len(trainset), len(valset),len(testset), vocab_size, n_classes))

 [학습셋] : 20000 [검증셋] : 5000 [테스트셋] : 25000 [단어수] : 46159 [클래스] : 2 


# RNN 모델 구현 
self.n_layers = n_layers # 은닉벡터 
self.embed = nn.Embedding(n_vocab, embed_dim)
n_vocab : 단어를 사전형태로 표현시 단어 수 
embed_dim : 임베딩된 단어 텐서가 지니는 차원값

self.hidden_dim = hidden_dim RNN을 통해 생성되는 은닉벡터 차원값
self.dropout = nn.Dropout(dropout_p) 드롭아웃
       
nn.RNN 은 입력이 길어지면 학습 도중 기울기가 너무 작어지거나 (vanishing gradent) 커져 ( gradient explosion)
앞부분에 대한 정보를 정확히 담지 X
-> 이러한 것을 보완한 것이 nn.gru

GRU 는 시계열 데이터 속 벡터 사이의 정보 전달량을 조절함으로써 기울기를 적정하게
유지하고 문장 앞부분의 정보가 끝까지 도달되도록 한다. 

update gate 이전 은닉 벡터가 지닌 정보를 새로운 벡터가 얼마나 유지할 지 정함
reset gate 새로운 입력이 이전 은닉 벡터와 어떻게 조합하는지 결정

self.gru = nn.GRU(embed_dim, self.hidden_dim, num_layers = self.n_layers, batch_first = True)


GRU도 RNN과 마찬가지로 시계열 데이터를 하나의 텐서로 압축 
텐서는 문장 전체에 대한 맥락을 담고 있는데 이 정보를 토대로 감정 분류를 하기 위해선
압축된 텐서를 신경망에 통과시켜 클래스에 대한 예측을 출력해야 한다.

self.out = nn.Linear(self.hidden_dim, n_classes)


x = self.embed(x) 워드 임베딩 ( 영화평드이 벡터의 배열로 변환 )
h_0 = self._init_state(batch_size = x.size(0)) 첫번째 은닉벡터 

RNN 계열 신경망은 입력 데이터 외에도 은닉벡터 h_0를 정의해 x와 함께 입력해야 한다. 
x, _ = self.gru(x, h_0) -> 은닉벡터들이 시계열 배열 형태로 반환
-> (batch_size,입력 x길이 , hidden_dim) 의 모양을 지닌 3d 텐서 
 
h_t = x[:,-1,:] 인덱싱을 통해 배치 내 모든 시계열 은닉 벡터들의 마지막 토큰들을 내포한 (batch_size,1 , hidden_dim) 모양의 텐서를 추출할 수 있음.
-> h_t 가 영화 리뷰 배열들을 압축한 은닉 벡터 

self.dropout(h_t)
logit = self.out(h_t) -> 신겨망 입력
return logit -> 출력

def _ init_state(self, batch_size = 1):
     weight = next (self.parameters()).data  -> 첫번째 가중치 텐서 추출 ( 모델 가중치와 같은 데이터 타입 )
     return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()
     
self.parameters() 는 신경망 모듈 (nn.Module)의 가중치 정보를 반복자 형태로 반환
반복자가 생성하는 원소들은 실제 신경망 가중치 텐서를 지님
new(self.n_layers, batch_size, self.hidden_dim) 모델의 가중치와 같은 모양으로 텐서로 반환
.zero_() 텐서 내 모든 값을 0으로 초기화 

-> 첫 번째 은닉 벡터는 보통 모든 특성 값이 0인 벡터로 설정 

In [15]:
class BasicGRU(nn.Module):
    def __init__ (self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p = 0.2 ):
        super(BasicGRU, self).__init__()
        print("Building Basic GPU model....")
        self.n_layers = n_layers # 은닉벡터 
        self.embed = nn.Embedding(n_vocab, embed_dim)# 사전에 있는 단어 수 
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)
        self.gru = nn.GRU(embed_dim, self.hidden_dim, num_layers = self.n_layers, batch_first = True)
        self.out = nn.Linear(self.hidden_dim, n_classes)
    
    def forward(self, x):
        x = self.embed(x)
        h_0 = self._init_state(batch_size = x.size(0))
        x, _ = self.gru(x, h_0)
        h_t = x[:,-1,:]
        self.dropout(h_t)
        logit = self.out(h_t)
        return logit
    
    def _init_state(self, batch_size = 1):
        weight = next (self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

batch.label은 1이나 2의 값을 가지고 있다
-> y.data.sub_(1) 로 깔끔하게 batch.label이 0 이나 1의 값을 가지게 한다.

In [18]:
def train(model, optimizer, train_iter):
    model.train()
    for b,batch in enumerate(train_iter):
        x,y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1)
        optimizer.zero_grad() #기울기를 0으로 초기화
             
        logit = model(x)  # 모델에 x를 입력해 예측값을 계산 
        loss = F.cross_entropy(logit,y)  #오차를 구한다.
        loss.backward()  #기울기 계산
        optimizer.step() #최적화 


In [19]:
def evaluate(model, val_iter):
    """evaluate model"""
    model.eval()
    corrects, total_loss = 0,0
    for batch in val_iter:
        x,y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1)
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction = 'sum')# reduction = 'sum'을 통해서 오차의 합을 구한다. 
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    
    size = len(val_iter.dataset)
    avg_loss = total_loss / size # 오차 평균 
    avg_accuracy = 100.0*corrects / size #정확도 평균 
    return avg_loss, avg_accuracy

# 모델 객체 정의

은닉 벡터 차원 -  256 
임베딩된 토큰 차원 - 128
최적화 알고리즘 - Adam


In [20]:
model = BasicGRU(1, 256, vocab_size, 128, n_classes, 0.5).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

Building Basic GPU model....


최종 모델은 학습 오차가 아닌 검증 오차가 최소화된 모델로 저장한다. 

In [ ]:
best_val_loss = None
for e in range(1, EPOCHS+1):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)
    
    print("[이폭: %d] 검증 오차 :%5.2f | 검증 ㄱ정확도: %5.2f" %(e,val_loss,val_accuracy))
    
    # 검증오차가 가장 적은 최적의 모델을 저장 
    if not best_val_loss or val_loss < best_val_loss :
        if not os.path.isdir("snapshot"):
            os.makedirs("snapshot")
        torch.save(model.state_dict(),'./snapshot/txtclassification.pt')
        best_val_loss = val_loss
        

In [ ]:
model.load_state_dict (torch.load('./snapshot/txtclassifivation.pt'))
test_loss, test_Acc = evaluate(model, test_iter)
print('테스트 오차:%5.2f | 테스트 정확도 : %5.2f' % (test_loss,test_acc))